In [15]:
import pandas as pd
import numpy as np
import plotly.express as px

'''
ROTH IRA - 6000
TRADITIONAL 401K - 19000
ROTH 401K - 19000
'''

'\nROTH IRA - 6000\nTRADITIONAL 401K - 19000\nROTH 401K - 19000\n'

In [18]:
def roth_401k(years, annual_contribution, percent_return):
    res = annual_contribution
    annual_list =[annual_contribution]
    for i in range(years):
        res = annual_contribution + annual_list[i] * percent_return
        annual_list.append(int(res))
    return annual_list

def traditional_401k_post_tax(df, tax_low):
    # input dataframe with 401k_roth column
    
    res=[]
    for i in df['401k_roth']:
        res.append(int(i*tax_low))
    return res

def tax_bracket(base_salary, annual_contribution):
    # if no tax bracket change, then go with roth 100% -- if bracket changes, put as much in traditional to lower tax bracket
 
    bracket_list = [9875, 40125, 85525, 163300, 207350, 518400]
    tax_list = [0.1, 0.12, 0.22, 0.24, 0.32, 0.35, 0.37]
    diff = base_salary - annual_contribution

    if diff > bracket_list[5]:
        traditional_contribution = 0
        roth_contribution = annual_contribution
    else:
        for i in range(4, 0, -1):
            if bracket_list[i] < diff <= bracket_list[i+1]:
                if base_salary - bracket_list[i+1] < 0:
                    tax_up = tax_list[i+1]
                    tax_low = tax_list[i]
                    traditional_contribution = 0
                    roth_contribution = annual_contribution
                    print('Your tax bracket is: {}%'.format(tax_up))
                    print('Contribute ${} to your traditional 401k'.format(traditional_contribution))
                    print('You should contribute {} to your roth 401k at your current tax rate of {}%'.format(roth_contribution, tax_up))
                else:
                    tax_up = tax_list[i+2]
                    tax_low = tax_list[i+1]
                    traditional_contribution = base_salary - bracket_list[i+1]
                    roth_contribution = annual_contribution - (base_salary - bracket_list[i+1])
                    print('Your upper tax is: {}%'.format(tax_up))
                    print('Your lower tax is: {}%'.format(tax_low))
                    print('You can lower your tax bracket to {}% by contributing to your traditional 401k: ${}'.format(tax_low, traditional_contribution))
                    print('Contribute the rest to your roth 401k: ${}'.format(roth_contribution))
    
    trad = round(traditional_contribution / 24, 2)
    roth = round(roth_contribution / 24, 2)
    print('-------------------------------------------------------------------------------------')
    print('Bimonthly traditional 401k contribution: {}'.format(trad))
    print('Bimonthly roth 401k contribution: {}'.format(roth))

    return traditional_contribution, roth_contribution, (1-tax_up), (1-tax_low)
    

def brokerage_list(years, annual_contribution, percent_return_non_401k, base_salary, tax_up, tax_low):
    roth = base_salary*tax_up - annual_contribution
    traditional = (base_salary-annual_contribution)*tax_low  
    difference = traditional - roth
    
    annual_trad_list =[difference]
    for i in range(years):
        res = difference + annual_trad_list[i] * percent_return_non_401k
        annual_trad_list.append(int(res))
    return annual_trad_list


def brokerage_post_tax(df):
    long_term_capital_gains_tax=0.2
    res=[]
    for i in df['lost roth 401k oppurtunity_cost']:
        res.append(int(i*(1-long_term_capital_gains_tax)))
    return res


'''
--------------------------------------------------------------------------------------------------
'''

def howdy_401k(age, retirement_age, annual_contribution, percent_return, percent_return_non_401k, base_salary):
    print('Your base salary is {}'.format(base_salary))
    years = retirement_age-age
    
    
    # make the table
    df = pd.DataFrame(list(range(years+1)), columns=['years'])
    
    # find tax range
    traditional_contribution, roth_contribution, tax_up, tax_low = tax_bracket(base_salary, annual_contribution)

    # create the return for 401k
    df['401k_roth'] = roth_401k(years, annual_contribution, percent_return)
    df['401k_traditional (pre_tax)'] = roth_401k(years, annual_contribution, percent_return)
    
    # create the return post tax traditional
    df['401k_traditional (post_tax)'] = traditional_401k_post_tax(df, tax_low)
    
    # create brokerage by getting the difference and capturing oppurtunity cost
    brokerage = brokerage_list(years, annual_contribution, percent_return, base_salary, tax_up, tax_low)
    df['lost roth 401k oppurtunity_cost'] = brokerage    
    
    # create brokerage returns post tax
    df['lost roth 401k oppurtunity_cost (capital_gain_tax)'] = brokerage_post_tax(df)
    
    df['difference (trad<>roth)'] = df['401k_roth'] - (df['401k_traditional (post_tax)']+df['lost roth 401k oppurtunity_cost (capital_gain_tax)'])
    
    return df

In [19]:
df=howdy_401k(age=26, retirement_age=65, annual_contribution=19000, percent_return=1.075, 
              percent_return_non_401k=1.08, base_salary=100000)
df

Your base salary is 100000
Your upper tax is: 0.24%
Your lower tax is: 0.22%
You can lower your tax bracket to 0.22% by contributing to your traditional 401k: $14475
Contribute the rest to your roth 401k: $4525
-------------------------------------------------------------------------------------
Bimonthly traditional 401k contribution: 603.12
Bimonthly roth 401k contribution: 188.54


,years,401k_roth,401k_traditional (pre_tax),401k_traditional (post_tax),lost roth 401k oppurtunity_cost,lost roth 401k oppurtunity_cost (capital_gain_tax),difference (trad<>roth)
0,0,19000,19000,14820,6180.0,4944,-764
1,1,39425,39425,30751,12823.0,10258,-1584
2,2,61381,61381,47877,19964.0,15971,-2467
3,3,84984,84984,66287,27641.0,22112,-3415
4,4,110357,110357,86078,35894.0,28715,-4436
5,5,137633,137633,107353,44766.0,35812,-5532
6,6,166955,166955,130224,54303.0,43442,-6711
7,7,198476,198476,154811,64555.0,51644,-7979
8,8,232361,232361,181241,75576.0,60460,-9340
9,9,268788,268788,209654,87424.0,69939,-10805


In [27]:
def run_m():
    age = int(input('Current age: '))
    retirement_age = int(input('Retirment age: '))
    annual_contribution = int(input('Contribution to your 401k: '))
    percent_return = float(input('Average annual return for your 401k: '))
    percent_return_non_401k = float(input('Annual return for brokerage: '))
    base_salary = int(input('Base Salary: '))

    return age, retirement_age, annual_contribution, percent_return, percent_return_non_401k, base_salary

In [29]:
age, retirement_age, annual_contribution, percent_return, percent_return_non_401k, base_salary = run_m()

Current age: 11
Retirment age: 66
Contribution to your 401k: 19000
Average annual return for your 401k: 1.075
Annual return for brokerage: 1.08
Base Salary: 100000


In [31]:
df1 = howdy_401k(age, retirement_age, annual_contribution, percent_return, percent_return_non_401k, base_salary)

Your base salary is 100000
Your upper tax is: 0.24%
Your lower tax is: 0.22%
You can lower your tax bracket to 0.22% by contributing to your traditional 401k: $14475
Contribute the rest to your roth 401k: $4525
-------------------------------------------------------------------------------------
Bimonthly traditional 401k contribution: 603.12
Bimonthly roth 401k contribution: 188.54


In [370]:
# df.to_csv('generic_35_year_plan', index=False)

57000.0

In [158]:
traditional=(100000-19000)*0.78
traditional

63180.0

In [159]:
roth = 100000*0.76 - 19000
traditional=(100000-19000)*0.78
difference = traditional - roth
difference

6180.0

In [ ]:
tax_rate = []
    
roth = base_salary*0.76 - 19000   #24% tax rate
traditional = (base_salary-19000)*0.78  #22% tax rate
difference = traditional - roth

In [96]:
df = pd.DataFrame(list(range(31)), columns=['years'])
res = 1000
annual_list =[1000]
for i in range(30):
    res = 1000 + annual_list[i] * 1.10
    annual_list.append(int(res))
df['return'] = annual_list
# df['post_tax'] = 
df

,years,return
0,0,1000
1,1,2100
2,2,3310
3,3,4641
4,4,6105
5,5,7715
6,6,9486
7,7,11434
8,8,13577
9,9,15934


In [39]:
for i in [1000] * 10:
    print(i*1.07)

1070.0
1070.0
1070.0
1070.0
1070.0
1070.0
1070.0
1070.0
1070.0
1070.0


In [50]:
annual_return = 1000
annual_list =[1000]
for i in range(9):
    annual_return += 1000 * 1.07
    annual_list.append(annual_return)

In [51]:
annual_return

10630.0

In [ ]:
def tax_bracket(base_salary):
    

In [306]:
# if no tax bracket change, then go with roth 100% -- if bracket changes, put as much in traditional to lower tax bracket
base_salary = 100000
annual_contribution = 15000

bracket_1 = 9875
bracket_2 = 40125
bracket_3 = 85525
bracket_4 = 163300
bracket_5 = 207350
bracket_6 = 518400
bracket_list = [bracket_1, bracket_2, bracket_3, bracket_4, bracket_5, bracket_6]
diff = base_salary - annual_contribution

if diff > bracket_6:
    traditional_contribution = 0
    roth_contribution = annual_contribution
else:
    for i in range(4, 0, -1):
        if bracket_list[i] < diff <= bracket_list[i+1]:
            if base_salary - bracket_list[i+1] < 0:
                traditional_contribution = 0
                roth_contribution = annual_contribution
            else:
                traditional_contribution = base_salary - bracket_list[i+1]
                roth_contribution = annual_contribution - (base_salary - bracket_list[i+1])
            
print('traditional: {}'.format(traditional_contribution))
print('roth: {}'.format(roth_contribution))

traditional: 14475
roth: 525


5
